In [1]:
import random
from datasets import load_dataset
from datasets import DatasetDict
import re
from transformers import AutoTokenizer

/root/miniconda3/envs/py3.10/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
dataset = load_dataset("jerryjalapeno/nart-100k-synthetic")
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['conversations', 'id'],
        num_rows: 99086
    })
})


In [3]:
tokenizer = AutoTokenizer.from_pretrained("llama-2-7b-chat-hf-phr_mental_therapy-3")

In [ ]:
# choose 30000 samples from the dataset
# dataset['train'] = dataset['train'].shuffle(seed=42).select(range(2000))
# print(dataset)

In [4]:
SYSTEM_PROMPT = """You are a helpful and joyous mental therapy assistant. Always answer as helpfully and cheerfully as possible, while being safe.  Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content.Please ensure that your responses are socially unbiased and positive in nature.\n\nIf a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information."""

def preprocessText(text):
    text = re.sub(r'Alex', '', text)
    text = re.sub(r'Charlie', '', text)
    # remove ", " when it appears at the start of a sentence
    text = re.sub(r'^, ', '', text)
    # remove " ." with "."
    text = re.sub(r' \.', '.', text)
    # remove " ," with ","
    text = re.sub(r' ,', ',', text)
    # remove " ?" with "?"
    text = re.sub(r' \?', '?', text)
    # remove ",." with "."
    text = re.sub(r',\.', '.', text)
    # remove ",?" with "?"
    text = re.sub(r',\?', '?', text)
    # remove more than one space
    text = re.sub(r' +', ' ', text)
    
    return text
    

def transform_dataset(data_row):
    id = data_row['id']
    data_row = data_row['conversations']
    for conv in data_row:
        if conv['from'] == 'human':
            conv['role'] = "user"
        elif conv['from'] == 'gpt':
            conv['role'] = "assistant"
        
        conv['content'] = preprocessText(conv['value'])
        del conv['from']
        del conv['value']
    
    system_dict = {}
    system_dict['role'] = "system"
    system_dict['content'] = SYSTEM_PROMPT
    data_row.insert(0, system_dict)
    # in conversational format the features name is "messages"
    return {"messages": data_row}

dataset = dataset.map(transform_dataset, remove_columns=['conversations'])

Map: 100%|██████████| 99086/99086 [00:47<00:00, 2096.64 examples/s]


In [5]:
# remove conversation with more than 1024 tokens, for training memory reasons.
dataset = dataset.map(lambda x: {"input_ids_length": len(tokenizer.apply_chat_template(x["messages"]))})
# filter out the samples that are too long
max_input_length = 1024
dataset = dataset.filter(lambda x: x["input_ids_length"] <= max_input_length)
dataset = dataset.remove_columns(["input_ids_length"])
print(dataset)

Filter: 100%|██████████| 99086/99086 [00:07<00:00, 12508.17 examples/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'messages'],
        num_rows: 31832
    })
})


In [6]:
# create a train_test_val split
train_test_split = dataset["train"].train_test_split(test_size=0.3)
train_val_split = train_test_split['test'].train_test_split(test_size=0.5)

train_test_val_split = DatasetDict({
    "train": train_test_split['train'],
    "test": train_val_split['train'],
    "val": train_val_split['test']
})

print(train_test_val_split)

DatasetDict({
    train: Dataset({
        features: ['id', 'messages'],
        num_rows: 22282
    })
    test: Dataset({
        features: ['id', 'messages'],
        num_rows: 4775
    })
    val: Dataset({
        features: ['id', 'messages'],
        num_rows: 4775
    })
})


In [8]:
print(train_test_val_split['train'][1]['messages'])

[{'content': "You are a helpful and joyous mental therapy assistant. Always answer as helpfully and cheerfully as possible, while being safe.  Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content.Please ensure that your responses are socially unbiased and positive in nature.\n\nIf a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.", 'role': 'system'}, {'content': "I'm shocked. I feel like I've lost all control over my personal relationships.", 'role': 'user'}, {'content': 'I hear the sadness in your words. It must be difficult to experience that loss of control. Can you share more about what led to this feeling?', 'role': 'assistant'}, {'content': "Failed relationships. They keep piling up one after another. I just can't seem to make them work.", 'role': 'user'}, {'content': "It s

In [9]:
train_test_val_split.push_to_hub("phr-mental-therapy-dataset-conversational-format-1024-tokens")

Uploading the dataset shards: 100%|██████████| 1/1 [00:02<00:00,  2.34s/it]


CommitInfo(commit_url='https://huggingface.co/datasets/vibhorag101/phr-mental-therapy-dataset-conversational-format-1024-tokens/commit/8d87e0a9adee986951b13146d4124dd9595cd136', commit_message='Upload dataset', commit_description='', oid='8d87e0a9adee986951b13146d4124dd9595cd136', pr_url=None, pr_revision=None, pr_num=None)